In [1]:
from meshmaker.plt import plot
from PIL import Image


img = Image.new('RGBA', [500, 500], 255)
data = img.load()

for x in range(img.size[0]):
    for y in range(img.size[1]):
        data[x,y] = (
            x % 255,
            y % 255,
            0,#(x**2-y**2) % 255,
            255,
        )
img.save('./resources/textures/dummyimage2.png')

texture_image = Image.open('./resources/textures/dummyimage2.png')
        
f, ax = plot()
ax.imshow(texture_image)


In [15]:
from pywavefront import visualization
from pywavefront.material import MaterialParser
import pyglet
from pyglet.window import key
from pyglet.gl import *     
from collections import defaultdict
import os
from meshmaker.vec3 import vec3
import numpy as np
np.seterr(all='raise')


class MMCamera:

    def __init__(self):
        self.mode = 3
        self.w, self.h = 640, 480
        self.far = 8192
        self.init()
    
    def init(self):
        self.x, self.y, self.z = 0, 0, 512
        #self.rx, self.ry, self.rz = 30, 45, 90
        self.rx, self.ry, self.rz = -60, 0, -60
        scale = 100
        self.sx, self.sy, self.sz = scale, scale, scale
        self.fov = 60
        self.apply_mode()

    def view(self, width, height):
        print(f'Viewport: {str(width)}, {str(height)}')
        self.w, self.h = width, height
        glViewport(0, 0, width, height)
        self.apply_mode()
        
    def apply_mode(self):
        if self.mode == 2:
            self.isometric()
        elif self.mode == 3:
            self.perspective()
        else:
            self.default()
            
    def default(self):
        glMatrixMode(GL_PROJECTION)
        glLoadIdentity()
        glOrtho(0, self.w, 0, self.h, -10, 10)
        glMatrixMode(GL_MODELVIEW)

    def isometric(self):
        glMatrixMode(GL_PROJECTION)
        glLoadIdentity()
        glOrtho(-self.w / 2, self.w / 2, -self.h / 2, self.h / 2, 0, self.far)
        glMatrixMode(GL_MODELVIEW)

    def perspective(self):
        glMatrixMode(GL_PROJECTION)
        glLoadIdentity()
        gluPerspective(self.fov, float(self.w) / self.h, 0.1, self.far)
        glMatrixMode(GL_MODELVIEW)
        
    def key(self, symbol, modifiers):
        if symbol == key.F1:
            self.mode = 1
            self.default()
        elif symbol == key.F2:
            self.mode = 2
            self.isometric()
        elif symbol == key.F3:
            self.mode = 3
            self.perspective()
        elif symbol == key.R:
            self.init()

    def drag(self, x, y, dx, dy, button, modifiers):
        if button == 1:
            self.x -= dx * 2
            self.y -= dy * 2
        elif button == 2:
            self.x -= dx * 2
            self.z -= dy * 2
        elif button == 4:
            self.ry += dx / 4
            self.rx -= dy / 4

    def scroll(self, x, y, scroll_x, scroll_y):
        if self.mode == 3:
            self.fov -= scroll_y
            self.apply_mode()

    def apply(self):
        glLoadIdentity()
        glTranslated(-self.x, -self.y, -self.z)
        glRotatef(self.rx, 1.0, 0.0, 0.0)
        glRotatef(self.ry, 0.0, 1.0, 0.0)
        glRotatef(self.rz, 0.0, 0.0, 1.0)
        glScalef(self.sx, self.sy, self.sz)


class MMMaterials(MaterialParser):

    default = """newmtl {0}
Ns 1.0\nKa 1.0 1.0 1.0\nKd 1.0 1.0 1.0
Ks 1.0 1.0 1.0\nKe 0.0 0.0 0.0
Ni 1.0\nd 1.0\nillum 10\nmap_Kd {1}"""
    
    def __init__(self, root='./resources/textures/'):
        super().__init__('./')
        
    def create_line_generator(self):
        lines = []
        for r, ds, fs in os.walk('./resources/textures/'):
            for f in fs:
                name = f[:f.rfind('.')]
                path = os.path.join(r, f)
                lines.extend(self.default.format(name, path).split('\n'))
        yield from lines


class MMWindow(pyglet.window.Window):
    
    materials = MMMaterials().materials
    print(f'found {len(materials)} materials: {tuple(materials.keys())}')
    
    def __init__(self, factory):
        super().__init__(resizable=True)
        self._wireframe = False
        self.opengl_init()
        self.update(factory)
        self.camera = MMCamera()
        self.on_mouse_drag = self.camera.drag
        self.on_mouse_scroll = self.camera.scroll
        pyglet.app.run()
    
    def opengl_init(self):
        glEnable(GL_BLEND)
        glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)
        glDepthFunc(GL_LEQUAL)

    def update(self, factory=None):
        self.lines = defaultdict(list)
        self.axes()
        if factory is not None:
            self.factory = factory
        if hasattr(self.factory, 'scene'):
            self.scene = self.factory.scene()
        else:
            self.scene = tform()
            print(f'Factory has no "scene" method: {self.factory}')
        for name, material in self.materials.items():
            material.gl_floats = None            
            material.vertices, material.wireframe = self.draw_material(self.scene, name)
            material.vertex_format = 'T2F_N3F_V3F' if material.vertices else ''
            if self._wireframe:
                self.lines['r'].extend(material.wireframe)
    
    def draw_material(self, tf, material, vertices=None, wireframe=None):
        """Aggregate vertex data from a scene graph for a material"""
        vertices = [] if vertices is None else vertices
        wireframe = [] if wireframe is None else wireframe
        if hasattr(tf, 'models'):
            for model in tf.models:
                for mesh in model.meshes.get(material, ()):
                    tridata, wiredata = mesh.T2F_N3F_V3F(tf)
                    vertices.extend(tridata)
                    wireframe.extend(wiredata)
        for child in tf.children:
            self.draw_material(child, material, vertices, wireframe)
        return vertices, wireframe

    def on_key_press(self, symbol, modifiers):
        self.camera.key(symbol, modifiers)
        if symbol == key.Q:
            print('Closing Window!')
            self.close()
        elif symbol == key.W:
            print('Toggle Wireframe!')
            self._wireframe = not self._wireframe
            self.update()
        else:
            parameters = getattr(self.factory, 'parameters', {})
            operation = parameters.get(symbol)
            if operation is not None:
                operation(symbol, modifiers)
                self.update()
                
    def on_resize(self, width, height):
        self.camera.view(width, height)
        
    def on_draw(self):
        self.clear()
        glClearColor(0.8, 1.0, 1.0, 0.1)
        self.camera.apply()        
        for name, material in self.materials.items():
            if material.vertex_format:
                visualization.draw_material(material)
        self.on_draw_lines()

    colors = dict(
        r=(1.0, 0.0, 0.0),
        g=(0.0, 1.0, 0.0),
        b=(0.0, 0.0, 1.0),
    )

    def on_draw_lines(self):
        glBegin(GL_LINES)
        for color, segments in self.lines.items():
            for u, v in segments:
                r, g, b = self.colors.get(color, color)
                glColor3f(r, g, b)
                glVertex3f(u.x, u.y, u.z)
                glVertex3f(v.x, v.y, v.z)
        if hasattr(self.factory, 'lines'):
            raise
            for color, segments in self.factory.lines.items():
                for u, v in segments:
                    r, g, b = self.colors.get(color, color)
                    glColor3f(r, g, b)
                    glVertex3f(u.x, u.y, u.z)
                    glVertex3f(v.x, v.y, v.z)
        glEnd()

    def axes(self, o=vec3.O()):
        self.lines['r'].append((o, o + vec3.X()))
        self.lines['g'].append((o, o + vec3.Y()))
        self.lines['b'].append((o, o + vec3.Z()))

    @classmethod
    def show(cls, factory):
        cls(factory)

found 29 materials: ('wood2', 'plywood1', 'wood', 'grass2', 'metal', 'brick1', 'roadline_continuous', 'hokiestone', 'indoor-cement', 'sidewalk2', 'concrete1', 'road', 'roadline', 'concrete2', 'asphalt', 'road2', 'brick2', 'concrete3', 'sidewalk1', 'roadline_w_continuous', 'tile3', 'tile5', 'tile6', 'tile2', 'tile4', 'tile1', 'dummyimage2', 'orangeboxtex', 'generic')


In [16]:
from meshmaker.base import Base
from meshmaker.model import Model
from meshmaker.tform import TForm
from meshmaker.quat import quat
from meshmaker.vec3 import vec3
from meshmaker.delaunay import triangulation
from meshmaker.geometry import batch, slide, loop_offset, loop_normal, isnear, near
from collections import defaultdict
import numpy as np


def loopO(loop):
    """Return a lexicographical loop origin index."""
    zs = [p.z for p in loop]
    return zs.index(min(zs))
        

class MeshAttribute(Base):
    """Lazy lookup of mesh related data"""
    
    def __init__(self, mesh, **kws):
        super().__init__(**kws)
        self.mesh = mesh
        self._lookup = {}

    def __getitem__(self, index):
        attribute = self._lookup.get(index)
        if attribute is None:
            attribute = self(index)
            self._lookup[index] = attribute
        return attribute

    def __call__(self, index):
        raise NotImplementedError
    
class FaceNormals(MeshAttribute):
    """Lazy lookup of face normals"""
    
    def __call__(self, f):
        vs = [self.mesh.vertices[v] for v in self.mesh.faces[f]]
        u, v, w = vs[0], vs[1], vs[2]
        N = (v - u).crs(w - u).nrm()
        return N
    
class FaceTangents(MeshAttribute):
    """Lazy lookup of face tangents"""
    
    def __call__(self, f):
        N = self.mesh.face_normals()[f] 
        T = vec3.X() if isnear(abs(N.z), 1) else N.crs(vec3.Z()).nrm()
        return T
    

class Mesh(Base):
    """2-manifold tri/quad mesh object"""
    
    def T2F_N3F_V3F(self, tf, flat=False):
        """Compute a T2F_N3F_V3F representation for rendering"""
        #self.dissolve()
        normals = self.vertex_normals()
        uvs = self.vertex_uvs()
        wireframe = []
        data = []            
        for f, face in self:
            ps, ns, us = [self.vertices[v] for v in face], normals[f], uvs[f]
            ps, ns = tf.transform(ps), tf.transform(ns)
            if len(ps) == 3:
                pass
            elif len(ps) == 4:
                ps.insert(3, ps[2]);ps.insert(3, ps[0])
                ns.insert(3, ns[2]);ns.insert(3, ns[0])
                us.insert(3, us[2]);us.insert(3, us[0])
            else:
                raise NotImplementedError
            for p, n, u in zip(ps, ns, us):
                data.extend([u.x, u.y, n.x, n.y, n.z, p.x, p.y, p.z])
            for u, v, w in batch(ps, 3):
                wireframe.extend([(u, v), (v, w), (w, u)])
        return data, wireframe

    def __init__(self):
        self.vertices = []
        self.faces = []
        self.e2f = {}
        self.v2f = defaultdict(set)
        self.meta = {}

    def __iter__(self):
        """Yield existing faces and their indices"""
        for f, face in enumerate(self.faces):
            if face is not None:
                yield f, face

    def face_normals(self):
        """Generate face normal vector lookup"""
        if getattr(self, '_face_normals', None) is None:
            self._face_normals = FaceNormals(self)
        return self._face_normals

    def face_tangents(self, normals=None):
        """Generate face tangent vector lookup"""
        if getattr(self, '_face_tangents', None) is None:
            self._face_tangents = FaceTangents(self)
        return self._face_tangents

    def vertex_normals(self, smooth=True):
        """Generate vertex normal vector lookup with optional smoothing"""
        normals = self.normals if getattr(self, 'normals', None) else {}
        self._face_normals = None
        face_normals = self.face_normals()
        for f, face in self:
            if normals.get(f) is None:
                if smooth:
                    ns = [[face_normals[o] for o in self.v2f[v]] for v in face]
                    ns = [vec3.sum(n).nrm() for n in ns]
                    #ns = [vec3.sum([face_normals[o] for o in self.v2f[v]]).nrm() for v in face]
                else:
                    ns = [face_normals[f]] * len(face)
                normals[f] = ns
        return normals

    def vertex_uvs(self):
        """Generate UV coordinate lookup possibly unwrapping missing faces"""
        uvs = self.uvs if getattr(self, 'uvs', None) else {}
        missing = []
        for f, face in self:
            if uvs.get(f) is None:
                missing.append(f)
        if missing:
            seams = self.perimeter(missing)
            while missing:
                f = missing.pop(0)
                if uvs.get(f) is None:
                    self.unwrap_uvs(f, seams=seams, uvs=uvs)
        return uvs

    def unwrap_uvs(self, f=None, O=None, X=None, Y=None, S=None, seams=None, uvs=None):
        """Recursively generate UV coordinate lookup via angle based flattening"""
        if f is None:
            for f, face in self:
                return self.unwrap_uvs(f, O, X, Y, S, seams, uvs)
        face_normals = self.face_normals()
        N = face_normals[f]
        O = (vec3.O(), vec3.O()) if O is None else O
        if X is None and Y is None:
            X = self.face_tangents()[f]
            Y = N.crs(X)
        elif X is None:
            X = N.crs(Y) # ??? might be flipping convention
        elif Y is None:
            Y = N.crs(X) # ??? might be flipping convention
        S = vec3.U() if S is None else S
        seams = set() if seams is None else seams
        uvs = {} if uvs is None else uvs
        locate = lambda p: vec3(O[1].x + S.x * (O[0].dot(X) - p.dot(X)),
                                O[1].y + S.y * (O[0].dot(Y) - p.dot(Y)),
                                O[1].z)
        uvs[f] = list(locate(self.vertices[v]) for v in self.faces[f])
        for i, j in slide(self.faces[f], 2):
            if not (i, j) in seams:
                adj = self.e2f.get((j, i))
                if adj is not None and adj not in uvs:
                    q = quat.uu(N, face_normals[adj])
                    self.unwrap_uvs(adj, O, X.cp().rot(q), Y.cp().rot(q), S, seams, uvs=uvs)
        return uvs

    def dissolve(self):
        """Merge duplicate vertices"""
        groups = defaultdict(list)
        for i, v in enumerate(self.vertices):
            for j in groups:
                if self.vertices[j].isnear(v):
                    groups[i].append(j)
                    print('dupe!')
                    break
            else:
                groups[i].append(i)
        raise NotImplementedError(f'{groups}')
    
    def _fp(self, p, e=0.00001):
        """Find vertex in neighborhood of p if one exists"""
        for i, o in enumerate(self.vertices):
            if p.isnear(o, e):
                return i

    def av(self, p, e=None):
        """Add vertex to the mesh without connectivity"""
        v = None if e is None else self._fp(p, e)
        if v is None:
            v = len(self.vertices)
            self.vertices.append(p)
        return v

    def findfaces(self, meta):
        """Find all faces with matching meta data"""
        return [f for f, face in self if (self.meta[f] == meta)]
    
    def af(self, loop, meta=None, e=0.00001):
        """Add a face to the mesh via a loop of vertices"""
        face = [(p if isinstance(p, int) else self.av(p, e=e)) for p in loop]
        f = len(self.faces)
        for i, j in slide(face, 2):
            self.v2f[i].add(f)
            self.e2f[(i, j)] = f
            self.meta[f] = meta
        self.faces.append(face)
        return f

    def rf(self, f):
        """Remove face f including its connectivity"""
        for i, j in slide(self.faces[f], 2):
            del self.e2f[(i, j)]
            self.v2f[i].remove(f)
        self.faces[f] = None
        del self.meta[f]
    
    def apy(self, py, e=0.00001, h=None, r=10000):
        """Add triangles covering a polygon, possibly with holes"""
        eloop, iloops = py                                                                    
        n = loop_normal(eloop)                                                                
        q = quat.toxy(n)                                                                      
        q.rot(eloop)                                                                          
        for iloop in iloops:                                                                  
            q.rot(iloop)                                                                      
        t = triangulation(py, e, h, r)                                                        
        q = q.fp()                                                                            
        q.rot(t.points)
        return [self.af(tri, e=e) for tri in t.simplices()]
    
    def fan(self, center, rim, e=0.00001):
        """Add triangles on surface of cone"""
        new_faces = []
        for u, v in slide(rim, 2, 0):
            new_faces.append(self.af(center, u, v, e=e))
        return new_faces
            
    def bridge(self, left, right, m=0, **kws):
        """Add quadrilaterals between a pair of loops"""
        assert len(left) == len(right)
        new_faces = []
        for (a, b), (c, d) in zip(slide(left, 2, m), slide(right, 2, m)):
            new_faces.append(self.af((a, b, d, c), **kws))
        return new_faces

    def bridges(self, loops, m=0, n=0, **kws):
        """Bridge a sequence of loops"""
        new_patches = []
        for u, v in slide(loops, 2, m):
            new_patches.append(self.bridge(u, v, n, **kws))
        return new_patches

    def grid(self, a, b, c, d, n=3, m=3, **kws):
        """Create grid patch over quadrilateral region"""
        rails = ([d] + d.line(c, n) + [c]), ([a] + a.line(b, n) + [b])
        lines = [([x] + x.line(y, m) + [y]) for x, y in zip(*rails)]
        return self.bridges(lines, m=1, n=1, **kws)
    
    def extrude(self, faces, offset):
        """Extrude a set of faces along a vector"""
        
        # the faces induce a set of seams
        # faces induce cover on 1-1 mappings of modified seams...
        # extrude each edge of each seam using some local/global strategy
        
        bottom = self.perimeter(faces)
        assert len(bottom) == 1, 'cannot extract surfaces with genus > 0'
        surface = [[self.vertices[v].cp() for v in self.faces[f]] for f in faces] 
        for f in faces:
            meta = self.meta[f]
            self.rf(f)
        patches = [[self.af(offset.trnps(new), meta=meta) for new in surface]]
        top = self.perimeter(patches[0])
        patches.append(self.bridge(
            [i for i, j in bottom[0]],
            [i for i, j in top[0]], meta=meta))
        return patches
    
    def opposite(self, i, j):
        """Select the opposing edge"""
        f = self.e2f.get((i, j))
        if f is None:
            return
        else:
            face = self.faces[f]
            k = face.index(i) - 2
            return face[k], face[k + 1]
    
    def vertexloop(self, i, j, loop=None):
        """Select a sequence of vertices without turning"""
        raise

    def edgeloop(self, i, j, loop=None):
        """Select a sequence of face-opposing edges"""
        if loop and (i, j) == loop[0]:
            return loop
        else:
            if loop is None:
                loop = []
            loop.append((i, j))
            far = self.opposite(j, i)
            return loop if far is None else self.edgeloop(*far, loop=loop)

    def edgesplit(self, i, j, a=0.5, e=0.00001):
        """Split the face incident to edge ij"""
        new_faces = []
        f = self.e2f.get((i, j))
        if f is not None:
            u = self.av(self.vertices[i].lerp(self.vertices[j], a), e=e)
            face = self.faces[f]
            meta = self.meta[f]
            self.rf(f)
            v = face.index(i) - 2
            if face[v] == j:
                new_faces.append(self.af((i, u, face[v + 1]), meta=meta))
                new_faces.append(self.af((u, j, face[v + 1]), meta=meta))
            else:
                w = face[v + 1]
                v = face[v]
                x = self.av(self.vertices[w].lerp(self.vertices[v], a), e=e)
                new_faces.append(self.af((i, u, x, w), meta=meta))
                new_faces.append(self.af((u, j, v, x), meta=meta))
        return new_faces
    
    def edgeloopsplit(self, i, j, a=0.5, loop=None):
        """Split the edges incident to each of an edgeloop ij"""
        split = []
        for u, v in self.edgeloop(i, j, loop=loop):
            split.extend(self.edgesplit(u, v, a))
        return split

    def facesplit(self, f, u, v, e=0.00001):
        """Break face in two based on intersection with uv"""
        raise
    
    def perimeter(self, faces):
        """Find the set of sequences of edges bounding a set of faces"""
        edges = []
        for f in faces:
            for i, j in slide(self.faces[f], 2):
                if (j, i) in edges:
                    edges.remove((j, i))
                else:
                    edges.append((i, j))
        seams = []
        if edges:
            seams = [[edges.pop(0)]]
            while edges:
                for k, (i, j) in enumerate(edges):
                    if i == seams[-1][-1][1]:
                        seams[-1].append((i, j))
                        edges.pop(k)
                        break
                else:
                    edges.pop(k)
                    seams.append([(i, j)])
        return seams
    

class ParamMesh(Base):
    """Proxy class for representing hierarchical mesh parameterizations"""
    
    def __init__(self, control, **kws):
        super().__init__(features=[], **kws)
        self.control = control
        self.showcontrol = False
        self.parameters = {
            key.C: self.on_toggle,
            key.UP: self.on_up, key.DOWN: self.on_down,
            key.LEFT: self.on_left, key.RIGHT: self.on_right,
            key.J: self.on_j, key.K: self.on_k,
        }

    def on_toggle(self, s, ms):
        self.showcontrol = not self.showcontrol
    def on_left(self, s, ms):
        vec3(0.9, 1, 1).sclps(self.control.vertices)
    def on_right(self, s, ms):
        vec3(1.1, 1, 1).sclps(self.control.vertices)
    def on_up(self, s, ms):
        vec3(1, 1.1, 1).sclps(self.control.vertices)
    def on_down(self, s, ms):
        vec3(1, 0.9, 1).sclps(self.control.vertices)
    def on_j(self, s, ms):
        vec3(1, 1, 0.9).sclps(self.control.vertices)
    def on_k(self, s, ms):
        vec3(1, 1, 1.1).sclps(self.control.vertices)
        
    def scene(self):
        if self.showcontrol:
            parts = [TForm(models=[Model(meshes={'dummyimage2': [self.control]})])]
        else:
            parts = [f(self.control, target) for f, target in self.features]
        return TForm(children=parts)


def cube_mesh(r=1):
    mesh = Mesh()
    bottom = [vec3(-r,-r,-r), vec3( r,-r,-r), vec3( r, r,-r), vec3(-r, r,-r)]
    top    = [vec3(-r,-r, r), vec3( r,-r, r), vec3( r, r, r), vec3(-r, r, r)]
    mesh.bridge(bottom, top, meta='side')
    mesh.af(top, meta='top')
    mesh.af(bottom[::-1], meta='bottom')
    return mesh


def cube_model(r=1, material='dummyimage2'):
    mesh = cube_mesh(r=r)
    model = Model()
    model.add(material, mesh)
    return model


def test():
    MMWindow.show(Base(scene=lambda : TForm(models=[cube_model()])))

test()

Viewport: 1918, 1059
Closing Window!


In [17]:
class SimpleMesh(ParamMesh):
    
    @staticmethod
    def vertexfeature(control, vertices):
        positions = [control.vertices[v] for v in vertices]
        models = [cube_model(0.2, 'orangeboxtex')]
        tfs = [TForm(position=p, models=models) for p in positions]
        return TForm(children=tfs)

    @staticmethod
    def edgefeature(control, edges):
        edges = [(control.vertices[i], control.vertices[j]) for i, j in edges]
        positions = [u.lerp(v, 0.3) for u, v in edges]
        models = [cube_model(0.1, 'orangeboxtex')]
        tfs = [TForm(position=p, models=models) for p in positions]
        return TForm(children=tfs)
    
    @staticmethod
    def facefeature(control, faces):
        mesh = Mesh()
        for f in faces:
            mesh.af([control.vertices[v] for v in control.faces[f]])
        return TForm(models=[Model(meshes={'orangeboxtex': [mesh]})])

    def __init__(self, control, **kws):
        super().__init__(control, **kws)
        points = (self.vertexfeature, ())
        segments = (self.edgefeature, ())
        surfaces = [f for f, face in control]
        surfaces = (self.facefeature, surfaces)
        self.features = [points, segments, surfaces]

In [18]:
class Facade(SimpleMesh):
    
    def __init__(self, control, **kws):
        super().__init__(control, **kws)
        
        layers = defaultdict(list)
        
        walls, roof = [], []
        for f, face in control:
            N = control.face_normals()[f]
            if isnear(N.z, 0):
                walls.append(f)
            elif N.z > 0:
                roof.append(f)
                
            layers[control.meta.get(f, f)].append(f)
                
        print(layers)
                
        trims = control.perimeter(roof)

        walls = (Wall(), walls)
        roof = (self.facefeature, roof)
        trims = [(trim, rim) for rim in trims]

        self.features = [walls, roof] + trims

        
def trim(control, edges):
    edges = [(control.vertices[i], control.vertices[j]) for i, j in edges]
    positions = [u.lerp(v, 0.3) for u, v in edges]
    models = [cube_model(0.1, 'wood2')]
    tfs = [TForm(position=p, models=models) for p in positions]
    return TForm(children=tfs)


class Wall(Base):
    
    __name__ = 'FuckingWall'
    
    def sections(self, loop, n=None, m=None):
        o = loopO(loop)
        w, h = loop[o].d(loop[o + 1]), loop[o].d(loop[o - 1])
        n = self.n if hasattr(self, 'n') else int(w / 1)
        m = self.m if hasattr(self, 'm') else int(h / 1)
        a, b, c, d = loop[o:] + loop[:o]
        rails = ([a] + a.line(b, n) + [b]), ([d] + d.line(c, n) + [c])
        lines = [([x] + x.line(y, m) + [y]) for x, y in zip(*rails)]
        for l, r in slide(lines, 2, 1):
            for (a, b), (c, d) in zip(slide(l, 2, 1), slide(r, 2, 1)):
                yield [b, a, c, d]
    
    def windowed(self, mesh, chunk):
        ww = 0.2
        wh = 0.2
        wz = 0.2
        o = loopO(chunk)
        cw, ch = chunk[o].d(chunk[o + 1]), chunk[o].d(chunk[o - 1])
        if True and cw > 1.1 * ww and ch > 1.1 * (wh + wz):
            T = (chunk[o + 1] - chunk[o]).nrm()
            x = chunk[o].lerp(chunk[o + 1], 0.5)
            portal = [(x - T*(ww/2)).ztrn(wz),
                      (x + T*(ww/2)).ztrn(wz),
                      (x + T*(ww/2)).ztrn(wz + wh),
                      (x - T*(ww/2)).ztrn(wz + wh)]
            mesh.bridge(chunk[o:] + chunk[:o], portal, e=0.00001)
        else:
            mesh.af(chunk, e=0.00001)
    
    def __call__(self, control, faces):
        mesh = Mesh()
        for f in faces:
            loop = [control.vertices[v] for v in control.faces[f]]
            for chunk in self.sections(loop):
                self.windowed(mesh, chunk)
        return TForm(models=[Model(meshes={'brick2': [mesh]})])


    
def test():
    cube = cube_mesh()
    
    split = cube.edgeloopsplit(0, 3, 0.4)
    
    sides = cube.findfaces('side')
    for i, f in enumerate(sides[::3]):
        cube.extrude([f], cube.face_normals()[f] * (i + 1))
    top = cube.findfaces('top')
    cube.extrude(top, cube.face_normals()[top[0]])
    cube._face_normals = None
    
    pm = Facade(cube)
    MMWindow.show(pm)

test()

defaultdict(<class 'list'>, {'bottom': [5], 'side': [7, 8, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28], 'top': [29, 30, 31, 32, 33]})
Viewport: 1918, 1059
Closing Window!


In [19]:
class MetaMesh(SimpleMesh):

    @staticmethod
    def textured(texture):
        def facefeature(control, faces):
            mesh = Mesh()
            for f in faces:
                mesh.af([control.vertices[v] for v in control.faces[f]])
            return TForm(models=[Model(meshes={texture: [mesh]})])
        facefeature.__name__ = texture
        return facefeature

    def __init__(self, control, **kws):
        super().__init__(control, **kws)
        features = defaultdict(list)
        methods = {}
        for f, face in control:
            meta = control.meta.get(f)
            if callable(meta):
                methods[meta.__name__] = meta
                features[meta.__name__].append(f)
        self.features = [(methods[m], faces) for m, faces in features.items()]

In [20]:
class Roof(Base):
    
    __name__ = 'FuckingRoof'
    
    def __init__(self, texture='tile6', **kws):
        super().__init__(**kws)
        self.texture = texture
        
    def __call__(self, control, faces):
        mesh = Mesh()
        
        seams = control.perimeter(faces)
        
        control._face_normals = None
        normals = control.face_normals()
        cliffs = []
        for seam in seams:
            cliff = []
            for i, j in seam:
                inside  = control.e2f.get((i, j))
                outside = control.e2f.get((j, i))
                if near(normals[inside].z - normals[outside].z, 0) > 0:
                    inside = [control.vertices[v] for v in control.faces[inside]]
                    outside = [control.vertices[v] for v in control.faces[outside]]
                    inside_z = vec3.com(inside).z
                    outside_z = vec3.com(outside).z
                    if inside_z > outside_z:
                        cliff.append((i, j))
            else:
                if cliff:
                    cliffs.append(cliff)

        rims = [self.cliffrailing(control, cliff) for cliff in cliffs]
        
        for f in faces:
            mesh.af([control.vertices[v] for v in control.faces[f]])
        return TForm(models=[Model(meshes={self.texture: [mesh]})], children=rims)

    def cliffrailing(self, control, cliff, w=0.02, h=0.2):
        complete = cliff[0][0] == cliff[-1][1]
        loop = [control.vertices[i] for i, j in cliff]
        if not complete:
            loop.append(control.vertices[cliff[-1][1]])

        loops = [loop]
        loops.append([p.cp().ztrn( h) for p in loops[-1]])
        loops.append(loop_offset(loops[-1], w, complete))
        loops.append([p.cp().ztrn(-h) for p in loops[-1]])
        
        mesh = Mesh()
        mesh.bridges(loops, m=1, n=(0 if complete else 1))
        return TForm(models=[Model(meshes={'orangeboxtex': [mesh]})])

In [21]:
class Entry(Base):

    __name__ = 'AnalEntry'

    def __init__(self, texture='plywood1', **kws):
        super().__init__(**kws)
        self.texture = texture
        
    def __call__(self, control, faces):
        return TForm(children=[self.door(control, f) for f in faces])
    
    def door(self, control, f):
        dw, dh, dz, dd = 0.3, 0.6, 0.02, 0.04
        
        loop = [control.vertices[v] for v in control.faces[f]]
        o = loopO(loop)
        w = loop[o + 1].d(loop[o])
        h = loop[o - 1].d(loop[o])
        dw = min(dw, 0.8 * w)
        dh = min(dh, 0.8 * h - dz)
        T = (loop[o + 1] - loop[o]).nrm()
        Z = (loop[o - 1] - loop[o]).nrm()
        x = loop[o].lerp(loop[o + 1], 0.5)
        a = x - T * (dw / 2) + Z * dz
        b = a + T * (dw)
        w, z = b + Z * dh, a + Z * dh

        N = control.face_normals()[f]
        loops = [loop[o + 1:] + loop[:o + 1], [b, w, z, a]]
        loops.append([p - N * dd for p in loops[-1]])
        
        mesh = Mesh()
        mesh.bridges(loops, m=1)
        #mesh.bridge(loop[o + 1:] + loop[:o + 1], [b, w, z, a])
        mesh.af(loops[-1])
        
        tf = self.access(x, control, control.faces[f][o], control.faces[f][o + 1])
        
        return TForm(models=[Model(meshes={self.texture: [mesh]})], children=[tf])
        
    def access(self, anchor, control, i, j):
        """Find a sequence of faces with sufficient steepness along edgeloop ji"""
        edges = control.edgeloop(i, j)
        edges.pop(0)

        height = 0
        direction = vec3.Z().crs(control.vertices[i] - control.vertices[j]).nrm()
        for i, j in edges:
            f = control.e2f.get((i, j))
            N = control.face_normals()[f]
            zs = [control.vertices[v].z for v in control.faces[f]]
            height += max(zs) - min(zs)
            if N.z > 0:
                break
        
        slope = 1.0
        sw = 0.5
        loop = []
        loop.append(anchor + vec3.nZ(height) + direction * (height / slope))
        loop.append(anchor.cp())
        loop.append(anchor.cp().ztrn(-height))

        N = loop_normal(loop)
        l = [(p + N * (sw / 2)) for p in loop]
        r = [(p - N * (sw / 2)) for p in loop]
        
        mesh = Mesh()
        mesh.af(l)
        mesh.af(r[::-1])
        mesh.bridge(r, l, m=2)
        
        return TForm(models=[Model(meshes={self.texture: [mesh]})])


In [22]:
def make_control_mesh():
    Z = vec3.Z()
    
    mesh = Mesh()
    
    grass = MetaMesh.textured('grass2')
    tile = MetaMesh.textured('tile2')
    foundation = MetaMesh.textured('tile3')
    wall = Wall()
    roof = Roof()
    entry = Entry()
    
    r = 2.5
    bottom = [vec3(-r,-r, 0), vec3( r,-r, 0), vec3( r, r, 0), vec3(-r, r, 0)]
    mesh.grid(*bottom, 4, 4, meta=grass)
    
    footprint = []
    for f, face in mesh:
        if all((mesh.e2f.get((j, i)) is not None) for i, j in slide(face, 2)):
            footprint.append(f)
            mesh.meta[f] = foundation
    
    #footprint.pop(-1)
    patches = mesh.extrude(footprint, Z * 0.2)
    for f in patches[0]:
        mesh.meta[f] = roof
            
    patches = mesh.extrude(patches[0], Z)
    for f in patches[1]:
        mesh.meta[f] = tile
    mesh.meta[patches[1][-5]] = entry
    
    patches[0].pop(-1)
    patches[0].pop(-2)
    patches = mesh.extrude(patches[0], Z)
    for f in patches[1]:
        mesh.meta[f] = wall
    
    patches[0].pop(-2)
    patches = mesh.extrude(patches[0], Z)
    for f in patches[1]:
        mesh.meta[f] = wall

    patches = mesh.extrude(patches[0], Z)
    for f in patches[1]:
        mesh.meta[f] = wall
    return mesh
    

def test():
    MMWindow.show(MetaMesh(make_control_mesh()))
    
test()

Viewport: 1918, 1059
Closing Window!


In [119]:
from meshmaker.vec3 import vec3
from meshmaker.quat import quat
from meshmaker.tform import TForm
from meshmaker.mesh import Trimesh
from meshmaker.model import Model
from meshmaker.geometry import loop_offset, loop_normal, isnear, near


class Window:
    """windows cover the seams between two wall meshes"""
        
    def jamb(seam, jamb_height, jamb_depth):
        seam = [p.cp() for p in seam]        
        a = vec3.Y(jamb_depth).trnps(seam)
        b = loop_offset(a, jamb_height)
        c = vec3.Y(-2 * jamb_depth).trnps([p.cp() for p in b])
        d = loop_offset(c,-jamb_height)
        seams = [list(seam) for seam in zip(a, b, c, d)]
        
        mesh = Trimesh2()
        patches = mesh.bridges(seams, 0, 1)        
        mesh.normals = mesh.vertex_normals(smooth=False)
        mesh.uvs = {}
        for patch in patches:
            mesh.uvs = mesh.unwrap_uvs(patch[2], X=vec3.Y(), S=vec3(2, 2, 0),
                                       seams=mesh.perimeter(patch), uvs=mesh.uvs)

        model = Model()
        model.add('wood2', mesh)
        return model

    @classmethod
    def scene(cls, seam, jamb_height=0.01, jamb_depth=0.01):
        return TForm(models=[cls.jamb(seam, jamb_height, jamb_depth)])

    
class Wall:

    # parameterize the decimation and detailing of control mesh faces
    
    def surface(root):
        mesh = Trimesh2()
        loop = root.transform(root.loop)
        p, n = loop[0], loop_normal(loop)
        seams = [tf.transform(tf.loop) for tf in root.children]
        holes = []
        patches = []
        for tf in root.children:
            seam = tf.transform(tf.loop)
            hole = [lintp(s, s + n, p, n) for s in seam]
            patches.append(mesh.bridge(hole, seam))
            holes.append(hole)
        patches.append(mesh.apy((loop, holes)))
        
        vertical = lambda s: near(mesh.vertices[s[0]].dxy(mesh.vertices[s[1]]), 0) > 0
        uv_seams = mesh.perimeter(patches[-1])
        uv_seams = list(filter(vertical, uv_seams))
        
        mesh.normals = mesh.vertex_normals(smooth=False)
        mesh.uvs = {}
        mesh.unwrap_uvs(patches[-1][0], X=vec3.X(), Y=vec3.Z(), S=vec3(2, 2, 0),
                        seams=uv_seams, uvs=mesh.uvs)
                        #seams=mesh.perimeter(patch), uvs=mesh.uvs)
    
        model = Model()
        material = getattr(root, 'texture', 'orangeboxtex')
        model.add(material, mesh)
        return model, [s for s, c in zip(seams, root.children) if c.mode != 'FAKE']
        
    @classmethod
    def scene(cls, root):
        tf = TForm(models=[])
        for child in root.children:
            assert child.mode == 'WALL'
            model, seams = cls.surface(child)
            tf.models.append(model)
            for seam in seams:
                tf.children.append(Window.scene(seam))
            
        return tf
    

class Fancy:

    @staticmethod
    def box(w, h):
        return [vec3(-w / 2, 0, 0), vec3(w / 2, 0, 0),
                vec3(w / 2, 0, h), vec3(-w / 2, 0, h)]
    
    def __init__(self):
        w, h, d = 2, 1, 0.02
        
        left  = [p - vec3.Y(d) for p in self.box( 2, 1)]
        right = [p + vec3.Y(d) for p in self.box(-2, 1)]
        
        
        holes = [
            self.box(0.2, 0.6),
            self.box(0.2, 0.4),
        ]

        windows = [
            (0, vec3(0.2, 0.0, 0.1)),
            (0, vec3(0.4, 0.0, 0.1)),
            (1, vec3(0.75, 0.0, 0.2)),
        ]
        
        root = TForm()
        
        left = TForm(loop=left, parent=root, mode='WALL', texture='orangeboxtex')
        for hole, o in windows:
            p = left.loop[0].lerp(left.loop[1], o.x) + vec3(0, o.y + d, o.z)
            TForm(loop=holes[hole], position=p, parent=left, mode='WINDOW')

        right = TForm(loop=right, parent=root, mode='WALL', texture='brick1')
        for hole, o in windows:
            p = right.loop[0].lerp(right.loop[1], 1 - o.x) + vec3(0, o.y - d, o.z)
            TForm(loop=holes[hole], position=p, rotation=quat.av(np.pi, vec3.Z()), parent=right, mode='FAKE')
        
        self.root = root
        
    def scene(self):
        return Wall.scene(self.root)

MMWindow.show(Fancy())

NameError: name 'Trimesh2' is not defined